#  PostgreSQL

In [ ]:
sudo apt update
sudo apt install postgresql postgresql-contrib
sudo service postgresql start
psql -U postgres


In [ ]:
CREATE DATABASE mydatabase;


In [ ]:
CREATE USER myuser WITH PASSWORD 'mypassword';
GRANT ALL PRIVILEGES ON DATABASE mydatabase TO myuser;


In [ ]:
pip install psycopg2-binary


In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'mydatabase',
        'USER': 'myuser',
        'PASSWORD': 'mypassword',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}


In [ ]:
Create and Migrate Django Models;
python manage.py makemigrations
python manage.py migrate


# Backup and Restore PostgreSQL

In [ ]:
"""Backup PostgreSQL Database: To create a backup of your database"""
pg_dump -U myuser mydatabase > mydatabase_backup.sql

"""Restore PostgreSQL Database: To restore a backup to a new or existing database"""
psql -U myuser mydatabase < mydatabase_backup.sql



# Perform Maintenance Tasks

In [ ]:
"""
Optimize Your Database:
Regularly run maintenance tasks like VACUUM to optimize your database
"""
VACUUM FULL;

"""
Check for Long Queries:
Use PostgreSQL’s tools to monitor and analyze long-running queries
"""
SELECT * FROM pg_stat_activity WHERE state = 'active';


# How to have better Query: 

Where to Write Queries Summary:

In Views (views.py) for data fetching and processing for your web pages.

In Django Shell to test and experiment with queries.

In Custom Management Commands for scripts and scheduled tasks.

In Admin (admin.py) to customize data display in the Django admin interface.

In Tests (tests.py) to write queries and check correctness in your automated tests.

# select_related and prefetch_related

Avoid N+1 Queries with select_related and prefetch_related:
For foreign key and many-to-many relationships, Django can reduce the number of queries using these methods.

In [ ]:
"""
select_related():
Use for foreign key relationships. 
It performs a SQL join and fetches related objects in a single query.
"""

# Fetch posts along with their authors in one query
Post.objects.select_related('author')


"""
prefetch_related(): 
Use for many-to-many relationships or reverse foreign key lookups.
It performs separate queries and joins in Python
"""
# Prefetch related tags for each post
Post.objects.prefetch_related('tags')


# Using Aggregations and Annotations

Django ORM supports database-level aggregations, which can help optimize operations like counting, summing, averaging, etc.

In [ ]:
from django.db.models import Count, Avg

# Count the number of posts
Post.objects.aggregate(total_posts=Count('id'))

# Calculate average post length
Post.objects.aggregate(average_length=Avg('content__length'))

#Aggregation Example: 

from django.db.models import Count, Avg

# Count the number of posts
Post.objects.aggregate(total_posts=Count('id'))

# Calculate average post length
Post.objects.aggregate(average_length=Avg('content__length'))


#Annotation Example:

# Annotate posts with their comment counts
Post.objects.annotate(comment_count=Count('comments'))


# Using PostgreSQL-Specific Features


In [ ]:
"""
ArrayField:
Django provides ArrayField to store lists of values (e.g., a list of tags
"""
from django.contrib.postgres.fields import ArrayField

class Post(models.Model):
    tags = ArrayField(models.CharField(max_length=100))

"""
Full-Text Search:
You can perform full-text searches using PostgreSQL's full-text search functionality.
"""
from django.contrib.postgres.search import SearchVector, SearchQuery, SearchRank

# Perform a full-text search on the 'title' and 'content' fields
Post.objects.annotate(search=SearchVector('title', 'content')).filter(search=SearchQuery('Django'))

"""
JSONField:
Store and query JSON data directly in the database using JSONField.
"""
from django.contrib.postgres.fields import JSONField

class Data(models.Model):
    json_data = JSONField()

# Querying JSON field
Data.objects.filter(json_data__name='John')



# Cashing : 

In [ ]:
from django.core.cache import cache

# Set a value in the cache
cache.set('my_key', Post.objects.all(), timeout=60*15)

# Get a value from the cache
posts = cache.get('my_key')

In [ ]:
"""
If you only want to check the existence of records,
use exists() instead of count() to improve performance.
"""
# Checking if any post exists
if Post.objects.filter(author=user).exists():
    print("User has posts")

"""
Use the distinct() method to avoid fetching duplicate records.
"""
Post.objects.distinct('author')


# Raw SQL for Complex Queries

In [ ]:
Post.objects.raw('SELECT * FROM myapp_post WHERE content LIKE %s', ['%Django%'])
